# SOGOS Overview for MLGeo 2022

Southern Ocean Glider Observations of the Submesoscale


Notes -
Updated 2022.10.26
- Recommend using with gp_659 and gp_660 from OneDrive
- Check project_overview.md in ml-geo git folder for additional notes
- First paper from SOGOS program: [Link to Dove (2021)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2021JC017178)


---
**Modules**

                sgmod_main as sg                      ytd, profile splitting... 
                sgmod_L3proc as gproc                 grid Xarray processing - buoyancy, AOU, transform, dav-BFSLE
                sgmod_DFproc as dfproc                DataFrame processing - season, dav-MLD, T-S binning, ESPER
                sgmod_plotting as sgplot              plotting functions 

---
**Data/Variable Names**
    
*Satellite:*    
 
                fsle_backwards                  map of daily FSLE snapshots
                satellite_data                  adt, geo velocities, ssh anomalies 
*Glider:*    
 
                gp_659                          L3 dataset gridded on pressure
                gi_659                          L3 dataset gridded on isopycnals
                dav_659                         L3 dive-averaged metrics, incl. MLD
                df_659                          Flattened dataframe for variable operations
    
 

In [6]:
# Import packages 
import xarray as xr
import pandas as pd
import numpy as np
import gsw 
import matplotlib.pyplot as plt
from cmocean import cm as cmo
from datetime import datetime
import scipy
import glidertools as gt

import importlib
import sgmod_main as sg
import sgmod_L3proc as gproc     # for L3 grid processing
import sgmod_DFproc as dfproc    # for dataframe processing
import sgmod_plotting as sgplot  

# from sgmod_main import fsle, adt

In [8]:
file_path = '/Users/sangminsong/OneDrive - UW/Code/SOGOS/data/sogos_Apr19_L23/'
L3_659 = xr.open_dataset(file_path + 'sg659_level3.nc')
L3_660 = xr.open_dataset(file_path + 'sg660_level3.nc')

In [14]:
# Import glider data
# Change this to your file path 
file_path = '/Users/sangminsong/Library/CloudStorage/OneDrive-UW/Code/SOGOS/gridded-vars/'

# Pressure-gridded data to 1m for each glider
gp_659 = xr.open_dataset(file_path + 'gp_659_forMLGeo1026.nc')
gp_660 = xr.open_dataset(file_path + 'gp_660_forMLGeo1026.nc')

# Isopycnal-gridded data (to .001 sigma) for each glider
gi_659 = xr.open_dataset(file_path + 'gi_659_forMLGeo1026.nc')
gi_660 = xr.open_dataset(file_path + 'gi_660_forMLGeo1026.nc')


In [12]:
gp_659

<xarray.Dataset>
Dimensions:     (depth: 1001, nprof: 912)
Coordinates:
  * nprof       (nprof) int64 0 1 2 3 4 5 6 7 ... 905 906 907 908 909 910 911
  * depth       (depth) int64 0 1 2 3 4 5 6 7 ... 994 995 996 997 998 999 1000
Data variables:
    time        (depth, nprof) datetime64[ns] ...
    dive        (nprof) int16 ...
    lat         (depth, nprof) float32 ...
    lon         (depth, nprof) float32 ...
    speed       (depth, nprof) float32 ...
    P           (depth, nprof) float32 ...
    SA          (depth, nprof) float32 ...
    CT          (depth, nprof) float32 ...
    oxygen_raw  (depth, nprof) float32 ...
    sigma       (depth, nprof) float64 ...
    days        (depth, nprof) float64 ...
    buoyancy    (depth, nprof) float64 ...
    oxy_tcorr   (depth, nprof) float64 ...
    oxygen      (depth, nprof) float64 ...
Attributes:
    id:                              sg659_level3
    time_coverage_resolution:        PT1S
    geospatial_vertical_units:       meter
    geospatial_vertical_resolution:  meter
    geospatial_vertical_positive:    no
    platform:                        glider
    geospatial_lat_units:            degrees
    geospatial_lat_resolution:       seconds
    geospatial_lon_units:            degrees
    geospatial_lon_resolution:       seconds
    institution:                     Applied Physics Lab, University of Washi...
    license:                         These data may be redistributed and used...
    keywords_vocabulary:             NASA/GCMD Earth Science Keywords Version...
    references:                      http://data.nodc.noaa.gov/accession/0092291
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    Conventions:                     CF-1.6
    standard_name_vocabulary:        CF-1.6
    featureType:                     trajectoryProfile
    nodc_template_version:           NODC_NetCDF_Trajectory_Template_v0.9
    naming_authority:                edu.washington.apl
    creator_name:                    Luc Rainville
    creator_url:                     https://iop.apl.washington.edu
    creator_email:                   lucrain@uw.edu
    platform_id:                     SG659
    source:                          Seaglider SG659
    summary:                         SG659 SOGOS April 2019
    project:                         SOGOS April 2019
    glider:                          659
    mission:                         2
    seaglider_software_version:      66.12
    base_station_version:            3.0
    base_station_micro_version:      0
    quality_control_version:         1.12
    processing_level:                1.12
    instrument:                      aa4831 sbe41 wlbb2fl 
    title:                           Physical, chemical, and biological data ...
    keywords:                        Water Temperature, Conductivity, Salinit...
    geospatial_lon_min:              30.04640
    geospatial_lon_max:              67.79239
    geospatial_lat_min:              -53.45803
    geospatial_lat_max:              -45.64820
    time_coverage_start:             2019-04-30T13:58:31Z
    time_coverage_end:               2019-11-25T03:53:12Z
    time_coverage_duration:          P0000-00-208T13:54:40
    geospatial_vertical_min:         0.00
    geospatial_vertical_max:         1000.00
    date_created:                    2021-09-24T17:29:48Z
    date_modified:                   2021-09-24T17:29:48Z
    uuid:                            0440e19e-1d5d-11ec-9be7-acde48001122

In [15]:
gi_659

<xarray.Dataset>
Dimensions:   (nprof: 912, sigma: 1001)
Coordinates:
  * sigma     (sigma) float64 26.8 26.8 26.8 26.8 26.8 ... 27.8 27.8 27.8 27.8
  * nprof     (nprof) int64 0 1 2 3 4 5 6 7 ... 904 905 906 907 908 909 910 911
Data variables:
    time      (sigma, nprof) datetime64[ns] ...
    days      (sigma, nprof) float64 ...
    lat       (sigma, nprof) float32 ...
    lon       (sigma, nprof) float32 ...
    dive      (nprof) int16 ...
    SA        (sigma, nprof) float64 ...
    CT        (sigma, nprof) float64 ...
    oxygen    (sigma, nprof) float64 ...
    P         (sigma, nprof) float64 ...
    buoyancy  (sigma, nprof) float64 ...
    AOU       (sigma, nprof) float64 ...